## Analisis de Datos de Pinguinos

Mediciones de tamaño, observaciones de nidadas y proporciones de isótopos en la sangre de pingüinos Adelie, Chinstrap y Gentoo adultos en busca de alimento observados en islas del archipiélago Palmer cerca de la estación Palmer, en la Antártida. Los datos fueron recopilados y puestos a disposición por la Dra. Kristen Gorman y el Programa de Investigación Ecológica a Largo Plazo (LTER) de la Estación Palmer.

In [11]:
import empiricaldist
import janitor
import matplotlib.pyplot as plt
import numpy as np
import palmerpenguins

ModuleNotFoundError: No module named 'palmerpenguins'